In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/student-radar/paper/src

/content/drive/MyDrive/student-radar/paper/src


In [3]:
pwd

'/content/drive/MyDrive/student-radar/paper/src'

In [4]:
!pip install torch torchvision numpy h5py matplotlib scipy scikit-learn wandb

In [5]:
!pip install tensorflow==2.12.0 keras==2.12.0 tensorflow-addons==0.20.0


In [ ]:
# !pip install "numpy<2.0"

# Load model

In [12]:
import os
import sys

ROOT_DIR = "/content/drive/MyDrive/student-radar/paper/src"
sys.path.insert(0, os.path.join(ROOT_DIR, "models", "hyperparemeters"))
sys.path.insert(0, os.path.join(ROOT_DIR, "models", "trainers"))

import tensorflow as tf
import numpy as np
from cwgangp import CWGANGP
from cwgangp_original import CWGANGPOriginal
from model_cwgangp import build_generator as build_generator_student
from model_cwgangp import build_discriminator as build_discriminator_student
# from model_cwgangp_original import build_generator as build_generator_original
# from model_cwgangp_original import build_discriminator as build_discriminator_original

def load_model(weights_path, model_type="student", codings_size=100, input_shape=(1024, 16), label_dim=1):
    """
    Loads the specified CWGANGP variant and restores weights.

    Args:
        weights_path: Path to the saved weights file
        model_type: "student" (CWGANGP) or "original" (CWGANGPOriginal)
        codings_size: Latent dimension size, must be set to 100
        input_shape: Input shape for the discriminator
        label_dim: Label dimension (usually 1 for scalar labels)

    Returns:
        model: The reconstructed and restored CWGAN-GP model
    """

    if model_type == "student":
        generator = build_generator_student(codings_size, label_dim)
        discriminator = build_discriminator_student(input_shape, label_dim)
        model = CWGANGP(discriminator, generator, latent_dim=codings_size)
    elif model_type == "original":
        generator = build_generator_original(codings_size, label_dim)
        discriminator = build_discriminator_original(input_shape, label_dim)
        model = CWGANGPOriginal(discriminator, generator, latent_dim=codings_size)
    else:
        raise ValueError("model_type must be 'student' or 'original'")

    # Optional compile if you're going to resume training
    # model.compile(...) # optional if compiling to resume training

    dummy_noise = tf.random.normal((1, codings_size))
    dummy_label = tf.zeros((1, 1))
    _ = model([dummy_noise, dummy_label])

    model.load_weights(weights_path)
    print(f"[✓] Model weights loaded from: {weights_path}")
    return model

In [14]:
model = load_model(
    '/content/drive/MyDrive/student-radar/paper/checkpoints/student/model-9vdudyqm.h5',
    model_type="student",
    codings_size=100,
    input_shape=(1024, 16),
    label_dim=1
)

# Generate samples
noise = tf.random.normal((4, 100))
labels = tf.zeros((4, 1))  # Or any conditional value
samples = model.generator([noise, labels])
print(samples.shape)  # should be (4, 1024, 16)

[✓] Model weights loaded from: /content/drive/MyDrive/student-radar/paper/checkpoints/student/model-9vdudyqm.h5
(4, 1024, 16)


# Test Model